# Cell Segmentation

This notebook uses SAM to segment the given images and stores the segmentation information in a csv file for each image

## Segment Anything Model (SAM)

Download checkpoint files here: https://pypi.org/project/segment-anything-py/#model-checkpoints

This models requieres 16 GB of RAM (or VRAM) to work.


### Imports

In [1]:
import os
import sys
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0" # If you have more than one GPU, use this to select the one you want to use

import cv2
import numpy as np
import pandas as pd
import albumentations as A
from tqdm import tqdm
from matplotlib import pyplot as plt
from PIL import Image

sys.path.insert(0, "../../packages/python")
from data import utils as data_utils
from data import augmentation as data_augmentation
from models import cell_segmentation as segmentators

/home/nicolas/Documentos/UTN/INA/giar_ina_dev/.venv/lib/python3.12/site-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.6' (you have '2.0.4'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
2025-05-06 20:29:22.373803: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-06 20:29:22.380080: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746574162.387354  411033 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin c

In [2]:
import tensorflow as tf
print("GPU Available:", tf.config.list_physical_devices('GPU'))
print("cuDNN Enabled:", tf.test.is_built_with_cuda())

GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
cuDNN Enabled: True


### Global definitions

In [3]:
sys.path.insert(0, "../../")
from config import MODELS_PATH, CROPPED_PATH, MEDIA_PATH

# Specific paths
SAM_CHECKPOINT_PATH = os.path.join(MODELS_PATH, "sam/sam_vit_h_4b8939.pth")

# Select the devicce: 
# "cuda" : Will use the NVIDIA GPU
# "cpu" : Will use the... CPU
DEVICE_USE = "cuda"

### Load SAM model

In [4]:
cmg = segmentators.SAMCellMaskGenerator(SAM_CHECKPOINT_PATH, model_type = 'vit_h', device = DEVICE_USE)

### Apply segmentation to the whole
Using SAM to segment the images in IMAGE_PATH the output will be stored in separeted csv files in CSV_PATH

In [9]:
# Dataset selection
DATASET = 'onion_cell_merged'#'ina' # Change this to 'onion_cell_merged' if needed
DATASET_SECTION = 'train' # Change this to 'train', 'valida' or 'test' for 'onion_cell_merged', empty for 'ina'

IMAGE_PATH = os.path.join(MEDIA_PATH, 'images', DATASET, 'images', DATASET_SECTION)
CSV_PATH = os.path.join(CROPPED_PATH, DATASET, 'data', DATASET_SECTION)

In [10]:
# Apply segmentation to the whole
data_utils.dataset_cell_segmentation(cmg, IMAGE_PATH, CSV_PATH)

100%|██████████| 452/452 [20:42<00:00,  2.75s/it]
